## CHAPTER6 결정트리
> - 결정 트리(Decision Tree)는 분류와 회귀 작업 그리고 다중출력 작업도 가능한 머신러닝 알고리즘이다.
> - 매우 복잡한 데이터셋도 학습할 수 있는 강력한 알고리즘이다.
> - 랜덤 포레스트의 기본 구성 요소이기도 하다.

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [2]:
iris = load_iris()
X = iris.data[:,2:] # 꽃잎의 길이와 너비
y = iris.target

In [5]:
tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

![test](./img/결정트리.png)
- 루트 노트(root node) 깊이가 0인 맨 꼭대기의 노드에서 시작한다.
> - 꽃잎의 길이가 2.45 보다 짧은지 검사하고 만약에 짧으면 왼쪽 노드(깊이가 1인 노드)로 아니면 오른쪽 노드(깊이가 1인 노드)로 이동한다. 
> - 깊이가 1인 왼쪽 노드의 경우 더 이상의 가지치기를 하지 않은 노드로 리프 노드(leaf node)라 한다.
>> - 리프노드의 경우 더 이상의 검사를 하지 않음으로 setosa로 예측한다.
> - 오른쪽 노드의 경우 꽃잎의 길이가 2.45 보다 긴 경우로 오른쪽 자식 노드로 이동한다.
> - 깊이가 1인 오른쪽 노드는 리프노드가 아니라서 꽃잎의 너비가 1.75보다 작은지 추가적으로 검사한다.
>> - 깊이가 2인 리프노드를 보면 꽃잎 길이가 2.45 보다 작고 꽃잎 너비가 1.75 보다 작은 경우 Versicolor 아니면 virginica로 예측함을 알 수 있다.
> - 노드의 sample 속성은 해당 노드에 몇개의 훈련세트가 있는지 알려준다.
> - 노드의 value 속성은 노드에서 분류되어 각 클래스에 얼마나 많은 훈련 샘플이 있는지 알려준다.
> - 노드의 gini 속성은 불순도를 측정한다.
>> -  한 노드의 모든 샘플이 같은 클래스에 속해 있다면 이 노드를 순수(gini 점수가 0이다.)
> - ![test](./img/지니불순도.png)
- __결정 트리의 여러 장점 중 하나는 데이터 전처리가 거의 필요하지 않다는 점이다. 특히 특성의 스케일을 맞추거나 평균을 원점에 맞추는 작업이 필요하지 않다.__
- 사이킷런은 이진 트리만 만드는 CART 알고리즘을 사용한다. 그러므로 리프 노드 외의 모든 노드는 자식 노드를 두 개씩 가진다.
- ID3 같은 알고리즘은 둘 이상의 자식 노드를 가진 결정 트리를 만들 수 있다.
- 결정 트리는 매우 직관적이고 결정 방식을 이해하기 쉽다. 이런 모델을 화이트박스 모델이라 하고 반대로 랜덤포레스트나 신경망은 블랙박스 모델이다. 이 알고리즘들은 성능이 뛰어나고 예측을 만드는 연산과정을 쉽게 확인 할 수 있지만 왜 그런 예측을 만드는지는 쉽게 설명 할 수 없다.

### 클래스 확률 추정
- 결정 트리는 한 샘플이 특정 클래스 k에 속할 확률을 추정할 수도 있다. 
- 먼저 이 샘플에 대해 리프노드를 찾기 위해 트리를 탐색하고 그 노드에 있는 클래스 k의 훈련 샘플의 비율을 반환한다.

In [8]:
tree_clf.predict_proba([[5,1.5]]) # 꽃잎의 길이 5 너비가 1.5 일때 클래스 예측

array([[0.        , 0.90740741, 0.09259259]])

In [9]:
tree_clf.predict([[5,1.5]]) # Iris-Versicolor로 예측

array([1])

### CART 훈련 알고리즘
- 결정 트리를 훈련시키기 위해 CART(Classification And Regression Tree) 알고리즘을 사용한다.
> - 먼저 훈련 세트를 하나의 특성 k의 임계값 t(k)를 사용해 두 개의 서브셋으로 나눈다.
- CART 비용함수
> - ![TEST](./img/CART.png)
> - 여기서 MSE(X)/G(left/right)는 서브셋의 불순도
> - 여기서 MSE(X)/G(left/right)는 서브셋의 샘플수
- 훈련세트를 성공적으로 둘로 나누었다면 같은 방식으로 서브셋을 나누고 이런 식으로 계속 반복한다. 이 과정은 (max_depth 매개변수로 정의된) 최대 깊이가 되면 중지하거나 불순도를 줄이는 분할을 찾을 수 없을때 멈춘다.
- CART 알고리즘은 탐욕적 알고리즘으로 현재 단계의 분할이 몇 단계를 거쳐 가장 낮은 불순도로 이어질 수 있을지 없을지는 고려하지 않는다. 즉, 훙륭한 솔루션을 만들지만 최적이라는 보장은 없다.

### 계산의 복잡도
- 결정 트리를 탐색하기 위해서는 약 log(m)/log(2) 개의 노드를 거쳐야한다. 여기서 m은 데이터 수
- 각 노드는 하나의 특성값만 확인하기 때문에 예측에 필요한 전체 복잡도는 특성 수와 무관하다.
> - 만약 max_feature가 지정되었다면 모든 특성을 비교하므로 n x mlog(m)이 된다.
### 엔트로피
- 기본적으로 지니 불순도가 사용되지만 criterion 매개변수를 'entropy'로 지정하여 엔트로피 불순도를 사용할 수 있다.
- 여기서는 모든 메세지가 동일할 때 엔트로피가 0이된다. 머신러닝에서 불순도의 측정 방법으로 많이 사용
> - ![test](./img/엔트로피.png)
- 지니 불순도가 계산이 더 빠르지만 가장 빈도 높은 클래스를 한쪽 가리로 고립시키는 경향이 있는 반면 엔트로피는 조금 더 균형 잡힌 트리를 만든다.
### 규제 매개변수
- 결정 트리는 훈련 데이터에 대한 제약사항이 거의 없다.
- 제한을 두지 않으면 트리가 훈련 데이터에 매우 가깝게 맞추려해서 과대적합되기 쉽다
- 결정트리는 훈련되기 전에 파라미터 수가 결정되지 않기 때문에 이런 모델을 비파라미터 모델이라 부르며 모델 구조가 데이터에 맞춰져서 고정되지 않고 자유롭다.
> - 반대로 파라미터 모델은 미리 정의된 모델 파라미터 수를 가지므로 자유도가 제한되고 과대적합될 위험이 줄어들지만 과소적합될 위험은 커진다.
- 결정트리의 경우 과대적합을 피하기 위해 자유도를 제한하는데 max_depth 매개변수를 통해 이를 조절
> - min_samples_split(분할되기 위해 노드가 가져야 하는 최소 샘플 수)
> - min_samples_leaf(리프 노드가 가지고 있어야 할 최소 샘플 수)
> - min_weight_fraction_leaf(min_samples_leaf과 같지만 가중치가 부여된 전체 샘플 수에서의 비율)
> - max_features(각 노드에서 분할에 사용할 특성의 최대 수)
>> - min으로 시작하는 매개변수를 증가시키거나 max로 시작하는 매개변수를 감소시키면 모델에 규제가 커진다.
- 제한 없이 결정 트리를 훈련시키고 불필요한 노드를 가지치기 하는 알고리즘도 있다.
- 순도를 높이는 것이 통계적으로 큰 효과가 없다면 리프 노드 바로 위의 노드는 불필요 할 수 있다.
- 대표적으로 카이제곱 검정 같은 통계적 검정을 사용하여 우연히 향상된 것인지 추정한다.
> - p값에 따라 어떤 임계값(=0.05) 보다 높으면 불필요한 노드로 간주되고 그 자식 노드는 삭제
- 가지치기는 불필요한 노드가 모두 없어질 때까지 계속된다.

### 회귀

In [10]:
from sklearn.tree import DecisionTreeRegressor

In [11]:
tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X,y)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

- 분류 트리와 비슷하지만 주요한 차이는 각 노드에서 클래스를 예측하는 대신 어떤 값을 예측한다는 점이다.
- 각 영역의 예측값은 그 영역에 있는 타깃값의 평균이 된다.
- 불순도를 최소화하는 방향으로 분할하는 대신 평균제곱오차(MSE)를 최소화하도록 분할하는 것을 제외하고는 비슷하다.
-------
- 결정트리의 주된 문제는 훈련 데이터에 있는 작은 변화에도 매우 민감하다는 것이다.
